# OpenCV/Python Introduction

In [1]:
# Start by installing opencv and dlib with pip/pip3:
# pip3 install opencv-python
# pip3 install dlib

In [2]:
# Start by importing opencv, numpy, an dlib
import cv2
import numpy as np
import dlib
import imutils
from imutils import face_utils
from math import atan2
from math import pi as PI

SHOW_FACE_POINTS = False

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('C:/Users/Kyle/Downloads/shape_predictor_68_face_landmarks.dat') # Replace with your path, 
                    # download here: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2, extract
moustache_mask = 1-(cv2.imread('C:/Users/Kyle/Desktop/mous.jfif')/255) # convert to mask, https://www.bing.com/th?id=OIP.ftDeHYROqGLXXU2IH3xg7wHaCd&pid=Api&rs=1

Converting Between Color Spaces

In [3]:
# This function will convert an image to greyscale by first converting it to the HSV
# colorspace, from BGR, then taking the Luminance Channel.
def ConvertToBW(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    grey = hsv[:,:,2]
    return grey

In [4]:
# This function will blur the image by average neighboring values
def BlurImage(image):
    kernel = np.array([[1,1,1,1,1],
                      [1,1,1,1,1],
                      [1,1,1,1,1],
                      [1,1,1,1,1],
                      [1,1,1,1,1]])
    kernel = kernel / 25
    blurred = cv2.filter2D(image, -1, kernel) # -1 keeps depth/percision the same
    return blurred

In [5]:
# This function will find edges in an image, using the Sobel operator/kernel
def DetectEdges(image):
    image = image.astype(np.float32)
    vertical_edge_kernel = np.array([[-1, 0, 1],
                                    [-2, 0, 2],
                                    [-1, 0, 1]])
    horizontal_edge_kernel = np.array([[-1, -2, -1],
                                       [0, 0, 0],
                                       [1, 2, 1]])
    vertical_edges = cv2.filter2D(image, -1, vertical_edge_kernel)
    horizontal_edges = cv2.filter2D(image, -1, horizontal_edge_kernel)
    summed = (vertical_edges ** 2) + (horizontal_edges ** 2)
    edges = summed ** 0.5
    edges[edges < 125] = 0 # can adjust threshold
    edges[edges >= 125] = 255
    return edges.astype(np.uint8)

In [6]:
# This function looks for a face
def Moustache(image):
    grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # OpenCV has a lot of color spaces, not just hsv
    rects = detector(grey, 1) # get bounding boxes of faces
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(grey, rect)
        shape = face_utils.shape_to_np(shape)
        
        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)], then draw the face bounding box
        (x, y, w, h) = face_utils.rect_to_bb(rect)

        if SHOW_FACE_POINTS:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # show the face number
            cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # loop over the (x, y)-coordinates for the facial landmarks
            # and draw them on the image
            for (x, y) in shape:
                cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
                
        # draw mustache
        nose_avg = np.average(shape[32:37], axis=0) # Slices exclusive, see https://www.pyimagesearch.com/wp-content/uploads/2017/04/facial_landmarks_68markup-768x619.jpg
        top_lip_avg = np.average(shape[51:54], axis=0)
        mous = imutils.resize(moustache_mask, width=int(w/3))
        
        # Calculate position
        (avg_y,avg_x) = (nose_avg + top_lip_avg)/2
        avg_x = int(avg_x)
        avg_y = int(avg_y)
        
        # Calculate rotation, using corners of mouth
        (left_x, left_y) = shape[54]
        (right_x, right_y) = shape[48]
        deltaY = left_y - right_y
        deltaX = left_x - right_x
        angleInDegrees = atan2(deltaY, deltaX) * 180 / PI
        mous = 1-imutils.rotate_bound(mous, angleInDegrees)
        
        lower_x = max(avg_x - int(mous.shape[0]/2), 0)
        higher_x = min(avg_x + int(mous.shape[0]/2), image.shape[0])
        lower_y = max(avg_y - int(mous.shape[1]/2), 1)
        higher_y = min(avg_y + int(mous.shape[1]/2), image.shape[1])
        #print(mous.shape)
        if (mous.shape[0] != higher_x - lower_x):
            higher_x = higher_x + mous.shape[0] - (higher_x - lower_x)
        if (mous.shape[1] != higher_y - lower_y):
            higher_y = higher_y + mous.shape[1] - (higher_y - lower_y)
        image[lower_x:higher_x, lower_y:higher_y, :] = mous * image[lower_x:higher_x, lower_y:higher_y, :]
    return image

In [7]:
# Create a VideoCapture object, pass 0 to read from (first) camera on system
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = imutils.resize(frame, width=300)
    
    if ret == True:
        # Display the resulting frame
        cv2.imshow('input',frame)
        cv2.imshow('BW',ConvertToBW(frame))
        cv2.imshow('Blur',BlurImage(frame))
        cv2.imshow('Edge',DetectEdges(frame))
        cv2.imshow('Moustache',Moustache(frame))

        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'): #25ms pause between each frame
            break

    # Break the loop
    else: 
        break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()